<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# Fourier-based Option Pricing

For several reasons, it is beneficial to have available alternative valuation and pricing approaches to the Monte Carlo simulation approach. One application area is to **benchmark Monte Carlo-based valuation results** against other (potentially more accurate) results. Another area is **model calibration to liquidly traded vanilla instruments** where generally faster numerial methods can be applied.

This part introduces **Fouried-based valuation functions** and benchmarks valuation results from the "standard", simulation-based DX Analytics modeling approach to output of those functions. 

In [1]:
import dx
import datetime as dt

## Risk Factors

The examples and benchmarks to follow rely on four different models:

* geometric Brownian motion (Black-Scholes-Merton 1973)
* jump diffusion (Merton 1976)
* stochastic volatility (Heston 1993)
* stochastic volatility jump diffusion (Bates 1996)

For details on these models and the Fourier-based option pricing approach refer to Hilpisch (2015) (cf. http://eu.wiley.com/WileyCDA/WileyTitle/productCd-1119037999.html).

We first define the single **market and valuation environments**.

In [2]:
# constant short rate
r = dx.constant_short_rate('r', 0.01)

In [3]:
# geometric Brownian motion
me = dx.market_environment('me', dt.datetime(2015, 1, 1))
me.add_constant('initial_value', 100.)
me.add_constant('volatility', 0.2)
me.add_constant('final_date', dt.datetime(2015, 12, 31))
me.add_constant('currency', 'EUR')

In [4]:
# jump component
me.add_constant('lambda', 0.4)
me.add_constant('mu', -0.6)
me.add_constant('delta', 0.2)

In [5]:
# stochastic volatiltiy component
me.add_constant('rho', -.5)
me.add_constant('kappa', 5.0)
me.add_constant('theta', 0.02)
me.add_constant('vol_vol', 0.3)

In [6]:
# valuation environment
val_env = dx.market_environment('val_env', dt.datetime(2015, 1, 1))
val_env.add_constant('paths', 55000)
    # 25,000 paths
val_env.add_constant('frequency', 'D')
    # weekly frequency
val_env.add_curve('discount_curve', r)
val_env.add_constant('starting_date', dt.datetime(2015, 1, 1))
val_env.add_constant('final_date', dt.datetime(2015, 12, 31))

In [7]:
# add valuation environment to market environment
me.add_environment(val_env)

Equipped with the single market environments and the valuation environment, we can instantiate the **simulation model objects**.

In [8]:
gbm = dx.geometric_brownian_motion('gbm', me)

In [9]:
jd = dx.jump_diffusion('jd', me)

In [10]:
sv = dx.stochastic_volatility('sv', me)

In [11]:
svjd = dx.stoch_vol_jump_diffusion('svjd', me)

## Plain Vanilla Put and Call Options

Based on the just defined risk factors, we define 8 diffent options---a **European put and call option per risk factor**, respectively.

In [12]:
# market environment for the options
me_option = dx.market_environment('option', dt.datetime(2015, 1, 1))
me_option.add_constant('maturity', dt.datetime(2015, 12, 31))
me_option.add_constant('strike', 100.)
me_option.add_constant('currency', 'EUR')
me_option.add_environment(me)
me_option.add_environment(val_env)

In [13]:
euro_put_gbm = dx.valuation_mcs_european_single('euro_put', gbm, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_gbm = dx.valuation_mcs_european_single('euro_call', gbm, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [14]:
euro_put_jd = dx.valuation_mcs_european_single('euro_put', jd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_jd = dx.valuation_mcs_european_single('euro_call', jd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [15]:
euro_put_sv = dx.valuation_mcs_european_single('euro_put', sv, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_sv = dx.valuation_mcs_european_single('euro_call', sv, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [16]:
euro_put_svjd = dx.valuation_mcs_european_single('euro_put', svjd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_svjd = dx.valuation_mcs_european_single('euro_call', svjd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

## Valuation Benchmarking

In this sub-section, we benchmark the **Monte Carlo value estimates** against the **Fourier-based pricing results**.

In [17]:
import numpy as np
import pandas as pd

We first define some parameters used throughout.

In [18]:
freq = '2m'  # used for maturity definitions
periods = 3  # number of intervals for maturity grid
strikes = 5  # number of strikes per maturity
initial_value = 100  # initial value for all risk factors
start = 0.8  # lowest strike in percent of spot
end = 1.2  # highest strike in percent of spot
start_date = '2015/3/1'  # start date for simulation/pricing

### Geometric Brownian Motion

We need to initialize the valuation object first.

In [19]:
euro_put_gbm.present_value()
  # method call needed for initialization

np.float64(7.423456)

There is a **valuation class for European put and call options in the Black-Scholes-Merton model** available called `BSM_european_option`. It is based on the analytical pricing formula for that model and is instantiated as follows:

In [20]:
bsm_option = dx.BSM_european_option('bsm_opt', me_option)

The following routine benchmarks the Monte Carlo value estimates for the **European put option** against the output from the valuation object based on the analytical pricing formula. The results are quite good since this model is quite easy to discretize exactly and therefore generally shows good convergence of the Monte Carlo estimates.

In [21]:
%%time
# European put
print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    bsm_option.maturity = maturity
    euro_put_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_put_gbm.update(strike=strike)
        mcs = euro_put_gbm.present_value()
        bsm_option.strike = strike
        ana = bsm_option.put_value()
        print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f '
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100))

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0318 |  0.0338 | -0.0020 |   -6.00 
0.244 |  90.000 |  0.6352 |  0.6524 | -0.0172 |   -2.64 
0.244 | 100.000 |  3.7792 |  3.8130 | -0.0338 |   -0.89 
0.244 | 110.000 | 10.6132 | 10.6957 | -0.0825 |   -0.77 
0.244 | 120.000 | 19.6991 | 19.8537 | -0.1546 |   -0.78 
0.411 |  80.000 |  0.1728 |  0.1748 | -0.0020 |   -1.15 
0.411 |  90.000 |  1.3110 |  1.3241 | -0.0131 |   -0.99 
0.411 | 100.000 |  4.8786 |  4.8985 | -0.0199 |   -0.41 
0.411 | 110.000 | 11.3475 | 11.4275 | -0.0801 |   -0.70 
0.411 | 120.000 | 19.9190 | 20.0325 | -0.1135 |   -0.57 
0.578 |  80.000 |  0.3852 |  0.3917 | -0.0066 |   -1.67 
0.578 |  90.000 |  1.9454 |  1.9466 | -0.0012 |   -0.06 
0.578 | 100.000 |  5.7118 |  5.7593 | -0.0475 |   -0.82 
0.578 | 110.000 | 12.0287 | 12.0934 | -0.0647 |   -0.54 
0.578 | 120.000 | 20.2406 | 20.3153 | -0.0747 |   -0.37 
CPU times: user 5.05 s, sys: 1.16 s, total: 6.2 s
Wall time: 6.29 s


The same now for the **European call option**.

In [22]:
euro_call_gbm.present_value()
  # method call needed for initialization

np.float64(8.415704)

In [23]:
%%time
# European calls
print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    euro_call_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_call_gbm.update(strike=strike)
        mcs = euro_call_gbm.present_value()
        bsm_option.strike = strike
        bsm_option.maturity = maturity
        ana = bsm_option.call_value()
        print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f ' \
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100))

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0754 | 20.2286 | -0.1533 |   -0.76 
0.244 |  90.000 | 10.7898 | 10.8716 | -0.0818 |   -0.75 
0.244 | 100.000 |  4.0305 |  4.0565 | -0.0260 |   -0.64 
0.244 | 110.000 |  0.9560 |  0.9636 | -0.0075 |   -0.78 
0.244 | 120.000 |  0.1483 |  0.1460 |  0.0023 |    1.56 
0.411 |  80.000 | 20.3666 | 20.5029 | -0.1362 |   -0.66 
0.411 |  90.000 | 11.6486 | 11.6932 | -0.0446 |   -0.38 
0.411 | 100.000 |  5.3208 |  5.3086 |  0.0122 |    0.23 
0.411 | 110.000 |  1.8716 |  1.8787 | -0.0070 |   -0.37 
0.411 | 120.000 |  0.5223 |  0.5246 | -0.0024 |   -0.45 
0.578 |  80.000 | 20.7761 | 20.8528 | -0.0768 |   -0.37 
0.578 |  90.000 | 12.4376 | 12.4654 | -0.0278 |   -0.22 
0.578 | 100.000 |  6.2575 |  6.3357 | -0.0782 |   -1.23 
0.578 | 110.000 |  2.7229 |  2.7274 | -0.0046 |   -0.17 
0.578 | 120.000 |  0.9962 |  1.0070 | -0.0108 |   -1.07 
CPU times: user 5.07 s, sys: 1.2 s, total: 6.27 s
Wall time: 6.38 s


### Benchmarking Function

All other valuation benchmarks are generated with **Fourier-based pricing functions** for which the handling is identical. We therefore use the following function for the benchmarks from now on:

In [24]:
def valuation_benchmarking(valuation_object, fourier_function):
    print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
    for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
        valuation_object.update(maturity=maturity)
        me_option.add_constant('maturity', maturity)
        for strike in np.linspace(start, end, strikes) * initial_value:
            T = (maturity - me_option.pricing_date).days / 365.
            valuation_object.update(strike=strike)
            mcs = valuation_object.present_value()
            me_option.add_constant('strike', strike)
            fou = fourier_function(me_option)
            print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.3f '
                % (T, strike, mcs, fou, mcs - fou, (mcs - fou) / fou * 100))

### Jump Diffusion

The next model is the jump diffusion as proposed by **Merton (1976)**.

In [25]:
euro_put_jd.present_value()
  # method call needed for initialization

np.float64(14.094397)

There is a Fourier-based pricing function available which is called `M76_put_value` and which is used for the benchmarking for the **European put options** that follows.

In [26]:
%time valuation_benchmarking(euro_put_jd, dx.M76_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1780 |  2.1594 |  0.0185 |   0.858 
0.244 |  90.000 |  3.2956 |  3.2826 |  0.0130 |   0.395 
0.244 | 100.000 |  5.8632 |  5.8842 | -0.0211 |  -0.358 
0.244 | 110.000 | 11.4922 | 11.6115 | -0.1193 |  -1.027 
0.244 | 120.000 | 19.9065 | 20.0857 | -0.1792 |  -0.892 
0.411 |  80.000 |  3.4588 |  3.4505 |  0.0082 |   0.239 
0.411 |  90.000 |  5.1195 |  5.2162 | -0.0967 |  -1.854 
0.411 | 100.000 |  8.2390 |  8.2266 |  0.0124 |   0.151 
0.411 | 110.000 | 13.3272 | 13.4430 | -0.1158 |  -0.861 
0.411 | 120.000 | 20.7956 | 20.9238 | -0.1282 |  -0.613 
0.578 |  80.000 |  4.5909 |  4.6090 | -0.0181 |  -0.392 
0.578 |  90.000 |  6.8705 |  6.8782 | -0.0077 |  -0.112 
0.578 | 100.000 | 10.3227 | 10.2077 |  0.1151 |   1.128 
0.578 | 110.000 | 15.1012 | 15.2251 | -0.1238 |  -0.813 
0.578 | 120.000 | 21.9732 | 22.0886 | -0.1154 |  -0.522 
CPU times: user 11.5 s, sys: 2.2 s, total: 13.7 s
Wall time: 13.9 s


Accordingly, the benchmarking for the **European call options** based on the Fourier-based `M76_call_value` function.

In [27]:
euro_call_jd.present_value()
  # method call needed for initialization

np.float64(15.207603)

In [28]:
%time valuation_benchmarking(euro_call_jd, dx.M76_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.2177 | 22.3543 | -0.1365 |  -0.611 
0.244 |  90.000 | 13.4144 | 13.5018 | -0.0874 |  -0.648 
0.244 | 100.000 |  6.0470 |  6.1277 | -0.0807 |  -1.318 
0.244 | 110.000 |  1.8965 |  1.8794 |  0.0171 |   0.909 
0.244 | 120.000 |  0.3857 |  0.3780 |  0.0078 |   2.056 
0.411 |  80.000 | 23.6193 | 23.7786 | -0.1593 |  -0.670 
0.411 |  90.000 | 15.5292 | 15.5853 | -0.0561 |  -0.360 
0.411 | 100.000 |  8.5977 |  8.6367 | -0.0390 |  -0.451 
0.411 | 110.000 |  3.8647 |  3.8941 | -0.0294 |  -0.755 
0.411 | 120.000 |  1.3992 |  1.4160 | -0.0167 |  -1.181 
0.578 |  80.000 | 24.9663 | 25.0701 | -0.1039 |  -0.414 
0.578 |  90.000 | 17.2940 | 17.3970 | -0.1030 |  -0.592 
0.578 | 100.000 | 10.7403 | 10.7841 | -0.0438 |  -0.406 
0.578 | 110.000 |  5.8329 |  5.8591 | -0.0263 |  -0.448 
0.578 | 120.000 |  2.7687 |  2.7803 | -0.0116 |  -0.418 
CPU times: user 11.5 s, sys: 2.18 s, total: 13.7 s
Wall time: 14 s


### Stochastic Volatility

Stochastic volatility models like the one of **Heston (1993)** are popular to reproduce implied volatility smiles observed in markets. First, the benchmarking for the **European put options** using the Fourier-based `H93_put_value` function.

In [29]:
euro_put_sv.present_value()
  # method call needed for initialization

np.float64(5.302358)

In [30]:
%time valuation_benchmarking(euro_put_sv, dx.H93_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0509 |  0.0504 |  0.0005 |   0.971 
0.244 |  90.000 |  0.5647 |  0.5741 | -0.0094 |  -1.633 
0.244 | 100.000 |  3.2527 |  3.3204 | -0.0677 |  -2.038 
0.244 | 110.000 | 10.1402 | 10.2404 | -0.1002 |  -0.979 
0.244 | 120.000 | 19.5829 | 19.7354 | -0.1524 |  -0.772 
0.411 |  80.000 |  0.1506 |  0.1603 | -0.0097 |  -6.048 
0.411 |  90.000 |  0.9533 |  1.0063 | -0.0530 |  -5.263 
0.411 | 100.000 |  3.9253 |  4.0234 | -0.0980 |  -2.437 
0.411 | 110.000 | 10.4258 | 10.5487 | -0.1229 |  -1.165 
0.411 | 120.000 | 19.5071 | 19.6293 | -0.1222 |  -0.622 
0.578 |  80.000 |  0.2460 |  0.2838 | -0.0379 | -13.345 
0.578 |  90.000 |  1.2763 |  1.3631 | -0.0868 |  -6.369 
0.578 | 100.000 |  4.3592 |  4.5467 | -0.1875 |  -4.125 
0.578 | 110.000 | 10.7197 | 10.8389 | -0.1191 |  -1.099 
0.578 | 120.000 | 19.4574 | 19.5767 | -0.1193 |  -0.609 
CPU times: user 10.4 s, sys: 2.75 s, total: 13.2 s
Wall time: 13.4 s


Second, the benchmarking for the **European call options** based on the Fourier-based `H93_call_value` function.

In [31]:
euro_call_sv.present_value()
  # method call needed for initialization

np.float64(6.314184)

In [32]:
%time valuation_benchmarking(euro_call_sv, dx.H93_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0882 | 20.2453 | -0.1571 |  -0.776 
0.244 |  90.000 | 10.6900 | 10.7933 | -0.1032 |  -0.956 
0.244 | 100.000 |  3.4756 |  3.5639 | -0.0884 |  -2.479 
0.244 | 110.000 |  0.5040 |  0.5083 | -0.0043 |  -0.838 
0.244 | 120.000 |  0.0260 |  0.0276 | -0.0016 |  -5.926 
0.411 |  80.000 | 20.3584 | 20.4884 | -0.1300 |  -0.634 
0.411 |  90.000 | 11.2658 | 11.3754 | -0.1095 |  -0.963 
0.411 | 100.000 |  4.3414 |  4.4335 | -0.0921 |  -2.077 
0.411 | 110.000 |  0.9477 |  0.9998 | -0.0521 |  -5.215 
0.411 | 120.000 |  0.1078 |  0.1214 | -0.0136 | -11.196 
0.578 |  80.000 | 20.6289 | 20.7450 | -0.1161 |  -0.560 
0.578 |  90.000 | 11.7303 | 11.8818 | -0.1515 |  -1.275 
0.578 | 100.000 |  4.9676 |  5.1231 | -0.1555 |  -3.035 
0.578 | 110.000 |  1.3639 |  1.4729 | -0.1090 |  -7.402 
0.578 | 120.000 |  0.2432 |  0.2684 | -0.0252 |  -9.388 
CPU times: user 10.5 s, sys: 2.73 s, total: 13.2 s
Wall time: 13.5 s


### Stochastic Volatility Jump-Diffusion

Finally, we consider the combination of the stochastic volatility and jump diffusion models from before as proposed by **Bates (1996)**. The Fourier-based pricing function for **European put options** is called `B96_put_value`.

In [33]:
euro_put_svjd.present_value()
  # method call needed for initialization

np.float64(12.896154)

In [34]:
%time valuation_benchmarking(euro_put_svjd, dx.B96_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1538 |  2.1638 | -0.0100 |  -0.462 
0.244 |  90.000 |  3.1485 |  3.2761 | -0.1276 |  -3.894 
0.244 | 100.000 |  5.4679 |  5.5889 | -0.1210 |  -2.165 
0.244 | 110.000 | 10.9542 | 11.0733 | -0.1190 |  -1.075 
0.244 | 120.000 | 19.6591 | 19.8344 | -0.1752 |  -0.883 
0.411 |  80.000 |  3.2698 |  3.4366 | -0.1667 |  -4.852 
0.411 |  90.000 |  4.9129 |  5.1339 | -0.2210 |  -4.305 
0.411 | 100.000 |  7.6325 |  7.7747 | -0.1422 |  -1.829 
0.411 | 110.000 | 12.3646 | 12.5744 | -0.2098 |  -1.668 
0.411 | 120.000 | 19.9550 | 20.1827 | -0.2278 |  -1.128 
0.578 |  80.000 |  4.3885 |  4.5480 | -0.1595 |  -3.507 
0.578 |  90.000 |  6.5286 |  6.7163 | -0.1877 |  -2.795 
0.578 | 100.000 |  9.3584 |  9.6585 | -0.3001 |  -3.107 
0.578 | 110.000 | 13.9213 | 14.1776 | -0.2563 |  -1.808 
0.578 | 120.000 | 20.6478 | 20.9423 | -0.2945 |  -1.406 
CPU times: user 17 s, sys: 3.85 s, total: 20.8 s
Wall time: 21.3 s


The Fourier-based counterpart function for **European call options** is called `B96_call_value`.

In [35]:
euro_call_svjd.present_value()
  # method call needed for initialization

np.float64(13.926504)

In [36]:
%time valuation_benchmarking(euro_call_svjd, dx.B96_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.1225 | 22.3587 | -0.2362 |  -1.057 
0.244 |  90.000 | 13.3309 | 13.4953 | -0.1643 |  -1.218 
0.244 | 100.000 |  5.7486 |  5.8325 | -0.0839 |  -1.439 
0.244 | 110.000 |  1.2880 |  1.3411 | -0.0531 |  -3.962 
0.244 | 120.000 |  0.1124 |  0.1266 | -0.0142 | -11.214 
0.411 |  80.000 | 23.4938 | 23.7647 | -0.2708 |  -1.140 
0.411 |  90.000 | 15.2886 | 15.5030 | -0.2145 |  -1.383 
0.411 | 100.000 |  7.9834 |  8.1848 | -0.2014 |  -2.460 
0.411 | 110.000 |  2.9062 |  3.0255 | -0.1193 |  -3.943 
0.411 | 120.000 |  0.6054 |  0.6749 | -0.0694 | -10.289 
0.578 |  80.000 | 24.7059 | 25.0091 | -0.3032 |  -1.212 
0.578 |  90.000 | 16.8964 | 17.2351 | -0.3386 |  -1.965 
0.578 | 100.000 |  9.9932 | 10.2349 | -0.2416 |  -2.361 
0.578 | 110.000 |  4.5448 |  4.8117 | -0.2669 |  -5.546 
0.578 | 120.000 |  1.4875 |  1.6340 | -0.1464 |  -8.960 
CPU times: user 16.9 s, sys: 3.78 s, total: 20.7 s
Wall time: 21.1 s


## Sources of Errors

Numerical methods like Monte Carlo simulation might suffer from different **sources of errors**, like for example:

* **discretization error**: every **discretization of a continuous time interval**---or a continuous state space to this end---leads to a so-called discretization error
* **approximation errors**: DX Analytics uses in several places approximative, **Euler-based discretization schemes** (e.g. for performance reasons and to allow for consistent correlation modeling) which are known to be biased
* **numerical errors**: the approximation of a continuous probability distribution by a **finite, discrete set of (pseudo-) random numbers** introduces also errors

**Copyright, License & Disclaimer**

© Dr. Yves J. Hilpisch | The Python Quants GmbH

DX Analytics (the "dx library" or "dx package") is licensed under the GNU Affero General
Public License version 3 or later (see http://www.gnu.org/licenses/).

DX Analytics comes with no representations or warranties, to the extent
permitted by applicable law.

[Learn More & Stay in Touch](https://linktr.ee/dyjh)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>